# Análise exploratória inicial dos dados de Expectativa de Vida

Equipe: Avelar Hostalácio, Carolina Brandão, Daniel Andrade, Enzo Pierazolli, Iago Gabino

Dataset utilizado para análises: [Life Expectancy WHO](https://www.kaggle.com/datasets/lashagoch/life-expectancy-who-updated)

## Leitura inicial do dataset e correção de colunas

Substituindo a coluna Economy_status_Developed para 'developed' booleana:

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats as ss

dataset = pd.read_csv('Life-Expectancy-Data.csv')

dataset.head()

Contando linhas e colunas:

In [ ]:
dataset.info()

## Reorganizando os dados do banco de dados

Dropamos algumas colunas que consideramos irrelevantes pro projeto e renomeamos outras, para facilitar a visualização/manipulação posterior dos dados.

In [ ]:
dataset.drop(['Thinness_ten_nineteen_years', 'Thinness_five_nine_years', 'Economy_status_Developing'], axis=1, inplace=True)

In [ ]:
##trocas developing e developed por true e false
dataset.rename(columns={'Economy_status_Developed':'developed'}, inplace=True)
dataset["developed"] = dataset["developed"].astype(bool)
dataset['developed'].unique()

In [ ]:
# renomear colunas com nomes com letra minuscula e no formato nome_da_coluna
dataset.rename(columns={'Country':'country', 'Region':'region', 'Year':'year', 'Infant_deaths':'infant_deaths', 'Under_five_deaths':'under_five_deaths', 'Adult_mortality':'adult_mortality', 'Alcohol_consumption':'alcohol_consumption', 'Hepatitis_B':'hepatitis_b', 'Measles':'measles', 'BMI':'bmi', 'Polio':'polio', 'Diphtheria':'diphtheria', 'Incidents_HIV':'incidents_hiv', 'GDP_per_capita':'gdp_per_capita', 'Population_mln':'population_mln', 'Schooling':'schooling', 'Life_expectancy':'life_expectancy'}, inplace=True)

In [ ]:
dataset.info()

#### Explicações breves sobre as colunas:

- **country:** país (179 países)

- **region:** região em que o país se encontra (África, América Central e Caribe, América do Norte, América do Sul, Ásia, 0ceania, Oriente Médio, União Europeia, e Resto da Europa)

- **year:** ano, de 2000 a 2015 

- **infant_deaths:** número de morte de bebês a cada 1000 habitantes

- **under_five_deaths:** número de morte de crianças menores de 5 anos a cada 1000 habitantes

- **adult_mortality:** número de mortes de adultos (15 a 60 anos) a cada 1000 habitantes

- **alcohol_consumption:** consumo de álcool puro (em litros) por pessoa com 15 anos ou mais

- **hepatitis_b:** porcentagem de cobertura de vacinação contra hepatite B em crianças de 1 ano

- **measles:** porcentagem de cobertura da primeira dose de vacinação contra sarampo em crianças de 1 ano

- **bmi:** índice de massa corporal médio de adultos. É definido como o peso de uma pessoa em quilogramas dividido pelo quadrado da altura dessa pessoa em metros (kg / m2)

- **polio:** porcentagem de cobertura de vacinação contra poliomielite em crianças de 1 ano

- **diphteria:** porcentagem de cobertura de vacinação contra difteria em crianças de 1 ano

- **incidents_hiv:** número de incidentes de HIV por 1000 habitantes com idade entre 15 e 49 anos

- **gdp_per_capita:** Produto Interno Bruto per capita (em USD)

- **population_mln:** população total em milhões

- **schooling:** média de anos que pessoas com 25 anos ou mais estudaram 

- **developed:** desenvolvido (true) ou em desenvolvimento (false)

- **life_expectancy:** média de expectativa de vida de ambos os sexos




## Plots para análise inicial de dados

In [ ]:
sns.lineplot(x = "year", y = "life_expectancy", hue = "developed", data = dataset)

In [ ]:
sns.lineplot(x = "year", y = "life_expectancy", hue = "developed", data = dataset)

In [ ]:
sns.lineplot(x = "year", y = "life_expectancy", data = dataset)

In [ ]:
f, ax = plt.subplots(figsize=(15, 8))
sns.scatterplot(x="gdp_per_capita", y="life_expectancy",hue="region", data= dataset)

In [ ]:
#plot an histogram
sns.histplot(x='life_expectancy', hue='developed', data=dataset)

In [ ]:
sns.pairplot(dataset, hue='developed', y_vars='life_expectancy', x_vars=['adult_mortality', 'infant_deaths', 'hepatitis_b', 'measles', 'bmi', 'under_five_deaths', 'polio', 'schooling', 'alcohol_consumption'])


In [ ]:
plt.figure(figsize=(10,10))

corr = dataset.corr(numeric_only=True)
mask = np.triu(np.ones_like(corr, dtype=bool))

sns.heatmap(dataset.corr(numeric_only=True), 
            mask=mask,
            center=0,
            annot=True,
            fmt='.2f',
            square=True,
            linewidths=.2,
            cbar_kws={"shrink": .6})

plt.title('Features Correlation Matrix Heatmap', fontsize=25);

## Consumo de Álcool e a Expectativa de Vida: Europa e África

A partir da EDA, observamos uma moderada correlação entre consumo de álcool e a expectativa de vida. Pensamos que uma maneira interessante de analisar essa correlação seria analisando as duas variáveis na União Europeia e na África, já que são continentes com realidades econômicas e sociais diferentes.

In [ ]:
europe_or_africa = dataset[(dataset['region'] == 'European Union') | (dataset['region'] == 'Africa')]

sns.scatterplot(x='alcohol_consumption', y='life_expectancy', data=europe_or_africa)


Calculando a correlação entre as duas variáveis nesses continentes:

In [ ]:
from scipy import stats

# calcula a correlação entre as variáveis
pearson_coef, p_value = stats.pearsonr(europe_or_africa['alcohol_consumption'], europe_or_africa['life_expectancy'])
print("Correlação de Pearson: ", pearson_coef, "\nP-valor da correlação: ", p_value)

Dado que a Correlação de Pearson entre as duas variáveis é aproximadamente 0.6, podemos dizer que há uma correlação positiva considerável. Além disso, um p-valor próximo de 0 indica que a correlação é estatisticamente significativa.

Porém esse resultado não parece condizer com a realidade, dado que o aumento do consumo de álcool não parece ser um fator que aumenta a expectativa de vida. Para entender melhor essa correlação, vamos analisar os dados de cada continente separadamente.

In [ ]:
sns.scatterplot(x='alcohol_consumption', y='life_expectancy', data=europe_or_africa, hue='region')

Calculando a correlação entre as duas variáveis na União Europeia e na África:

In [ ]:
eu = europe_or_africa[europe_or_africa['region'] == 'European Union']
af = europe_or_africa[europe_or_africa['region'] == 'Africa']

pearson_coef, p_value = stats.pearsonr(eu['alcohol_consumption'], eu['life_expectancy'])
print("Correlação de Pearson para Europa: ", pearson_coef, " | P-valor: ", p_value)

pearson_coef, p_value = stats.pearsonr(af['alcohol_consumption'], af['life_expectancy'])
print("Correlação de Pearson para Africa: ", pearson_coef, " | P-valor: ", p_value)

As correlações entre as variáveis nos continentes são negativas, e também são mais fracas, indicando que o consumo de álcool não é um fator muito relevante para a expectativa de vida nesses continentes, contrariando a análise conjunta dos continentes. Dessa forma, podemos dizer que a região é um _fator de confusão_ para a correlação entre as variáveis.

# Teste de Hipótese: Diferença entre o crescimento da expectativa de vida de países desenvolvidos e não desenvolvidos

H0: o país ser desenvolvido ou não não afeta a média da diferença entre a expectativa de vida de 2015 e 2000

H1: países não desenvolvidos tiveram um aumento médio maior na expectativa de vida entre 2000 e 2015 em comparação com países desenvolvidos

In [ ]:
# conta paises desenvolvidos
developed_countries = dataset[dataset['developed'] == True]
developed_countries['country'].nunique()

In [ ]:
# diferença entre a expectativa de vida de 2015 e 2000
# novo dataset com os países e as diferenças
life_expectancy_difference = pd.DataFrame(columns=['country', 'difference', 'region', 'developed'])
life_expectancy_difference['country'] = dataset['country'].unique()

# calcula a diferença para cada país
for country in life_expectancy_difference['country']:
    difference = dataset[(dataset['country'] == country) & (dataset['year'] == 2015)]['life_expectancy'].values[0] - dataset[(dataset['country'] == country) & (dataset['year'] == 2000)]['life_expectancy'].values[0]
    life_expectancy_difference.loc[life_expectancy_difference['country'] == country, 'difference'] = difference
    life_expectancy_difference.loc[life_expectancy_difference['country'] == country, 'region'] = dataset[dataset['country'] == country]['region'].values[0]
    life_expectancy_difference.loc[life_expectancy_difference['country'] == country, 'developed'] = dataset[dataset['country'] == country]['developed'].values[0]

life_expectancy_difference

In [ ]:
developed = life_expectancy_difference[life_expectancy_difference['developed'] == True]
not_developed = life_expectancy_difference[life_expectancy_difference['developed'] == False]


plt.hist(not_developed['difference'], bins=20, alpha=0.3, label='Não desenvolvidos', edgecolor='k')
plt.hist(developed['difference'], alpha=0.3, label='Desenvolvidos', edgecolor='k')
plt.xlabel('Diferença entre a expectativa de vida de 2015 e 2000')
plt.ylabel('Quantidade de países')
plt.legend()
plt.show()

In [ ]:
# diferença de médias entre os países desenvolvidos e não desenvolvidos
developed = life_expectancy_difference[life_expectancy_difference['developed'] == True]
not_developed = life_expectancy_difference[life_expectancy_difference['developed'] == False]

developed_mean = developed['difference'].mean()
not_developed_mean = not_developed['difference'].mean()

print('Média de diferença entre a expectativa de vida de 2015 e 2000 para países desenvolvidos: ', developed_mean)
print('Média de diferença entre a expectativa de vida de 2015 e 2000 para países não desenvolvidos: ', not_developed_mean)

t_obs = not_developed_mean - developed_mean

print('Diferença entre as médias: ', t_obs)

Testaremos se essa diferença foi devido ao acaso ou se é significativa, por meio do teste de permutação.

In [ ]:
df_shuffled = life_expectancy_difference.copy()
filtro = df_shuffled['developed'] == False
np.random.shuffle(filtro.values)
diff = df_shuffled[filtro]['difference'].mean() - df_shuffled[~filtro]['difference'].mean()
diff

Repetir o teste 10000 vezes:

In [ ]:
N = 10000
diferencas = np.zeros(N)
for i in range(N):
    np.random.shuffle(filtro.values)
    diff = df_shuffled[~filtro]['difference'].mean() - df_shuffled[filtro]['difference'].mean()
    diferencas[i] = diff

In [ ]:
plt.hist(diferencas, bins=50, edgecolor='k')
plt.xlabel('Diferença na Permutação')
plt.ylabel('Pr(diff)')
plt.vlines(t_obs, 0, 0.14, color='red')
plt.axvline(t_obs, color='red')
plt.show()

Cálculo do p-valor:

In [ ]:
p_value = np.count_nonzero(diferencas > t_obs) / N
p_value

#### CONCLUSÃO
Dado que o p-valor é menor que 0.05, podemos rejeitar a hipótese nula e concluir que a diferença entre a expectativa de vida de 2015 e 2000 foi significativamente maior em países não desenvolvidos.

## Teste de hipótese: Correlação entre Expectativa de vida e Educação


Será que a correlação, previamente calculada na matriz de correlações, de p=0.73 é dada pelo acaso? Ou uma taxa de educação maior realmente impacta positivamente na expectativa de vida da população?

In [ ]:
#correlação entre expectativa de vida e educação
corr = dataset['schooling'].corr(dataset['life_expectancy'])
print("Correlação de Pearson: ", corr)

In [ ]:
#calcular o p-value da correlação por permutação
N = 10000
dataset_shuffled = dataset.copy()
correlacoes = np.zeros(N)
for i in range(N):
    np.random.shuffle(dataset_shuffled['schooling'].values)
    correlacoes[i] = dataset_shuffled['schooling'].corr(dataset_shuffled['life_expectancy'])

plt.hist(correlacoes, bins=50, edgecolor='k')
plt.xlabel('Correlação na Permutação')
plt.ylabel('Densidade')
plt.axvline(corr, color='red')
plt.show()

É bastante claro, após a realização deste teste de permutação que a correlação entre escolaridade e expectativa de vida é realmente existente e tem p-value igual a zero. 

In [ ]:
#p-value
p_value = np.count_nonzero(correlacoes > corr) / N
p_value

# Análise experimental: Guerras

Primeiramente, procuramos países onde a expectativa de vida de 2015 foi menor que a de 2000.

In [ ]:
#países em que a diferença de expectativa de vida é negativa
life_expectancy_difference[life_expectancy_difference['difference'] < 0]

Iremos focar na Síria, pela maior diferença.

## SÍRIA

Analisando a expectativa de vida ao longo dos anos.

In [ ]:
# gráfico de expectativa de vida ao longo dos anos
syria = dataset[dataset['country'] == 'Syrian Arab Republic']
syria = syria.sort_values(by='year')
syria[['year', 'life_expectancy']]
plt.figure(figsize=(10, 5))
plt.xticks(np.arange(2000, 2016, 1.0))
plt.title('Expectativa de vida na Síria ao longo dos anos')
plt.plot(syria['year'], syria['life_expectancy'])
plt.xlabel('Ano')
plt.ylabel('Expectativa de vida em anos')

In [ ]:
# diferença entre a maior e a menor expectativa de vida da síria e anos em que ocorreram
maxYear = syria[syria['life_expectancy'] == syria['life_expectancy'].max()]['year']
minYear = syria[syria['life_expectancy'] == syria['life_expectancy'].min()]['year']
max = syria['life_expectancy'].max()
min = syria['life_expectancy'].min()
print('Ano com maior expectativa de vida: ', maxYear.values[0], ' | Expectativa de vida: ', max)
print('Ano com menor expectativa de vida: ', minYear.values[0], ' | Expectativa de vida: ', min)
print('Diferença: ', max - min)

A Síria é palco de uma guerra civil que começou em 2011 e ainda não acabou. A guerra civil na Síria é um conflito entre o governo sírio e forças da oposição, que começou com protestos pacíficos contra o governo sírio, liderado pelo presidente Bashar al-Assad. O conflito se transformou em uma guerra civil, após o uso da força pelo governo para reprimir os protestos. 

De acordo com o Observatório Sírio para os Direitos Humanos (OSDH), até 2021, a guerra civil causou exatas 494.438 mortes desde 2011. A grande maioria das mortes que a organização conseguiu confirmar aconteceu entre "o fim de 2012 e o fim de 2015". A seguir analisaremos as métricas relacionadas à mortalidade nesse período.

In [ ]:
#mortalidade infantil ao longo dos anos

plt.figure(figsize=(10, 5))
plt.xticks(np.arange(2000, 2016, 1.0))
plt.title('Mortalidade infantil na Síria ao longo dos anos')
plt.plot(syria['year'], syria['under_five_deaths'])
plt.xlabel('Ano')
plt.ylabel('Mortes de crianças menores de 5 anos a cada 1000 habitantes')

In [ ]:
#mortalidade adulta ao longo dos anos
plt.figure(figsize=(10, 5))
plt.xticks(np.arange(2000, 2016, 1.0))
plt.title('Mortalidade adulta na Síria ao longo dos anos')
plt.plot(syria['year'], syria['adult_mortality'])
plt.xlabel('Ano')
plt.ylabel('Mortes de adultos a cada 1000 habitantes')

É possível observar que de 2011 à 2014 o número de mortes infantis subiu uma dezena a cada 1000 habitantes, e desde 2007 o de adultos subiu mais de 70, o que condiz com a grande mortalidade apontada pela OSDH. A mortalidade adulta crescendo desde 2007 pode indicar que a guerra civil não foi o único fator que contribuiu para a queda da expectativa de vida na Síria. A queda aproximadamente a partir de 2013 pode indicar uma diminuição da intensidade da guerra, mas não é possível afirmar com certeza.


## IRAQUE

A Guerra do Iraque foi um conflito militar que teve início em 2003 e oficialmente terminou em dezembro de 2011. Essa guerra foi iniciada pelos Estados Unidos, liderados pelo presidente George W. Bush, que alegaram a existência de armas de destruição em massa no Iraque como justificativa para a invasão. O conflito resultou em um número significativo de mortes, e embora não haja um consenso definitivo sobre o número exato, estima-se que o conflito tenha causado a morte de centenas de milhares de pessoas, gerando uma grande devastação ao país e desencadeando consequências políticas e sociais que persistem até os dias atuais.

Analisando a expectativa de vida ao longo dos anos.

In [ ]:
#gráfico de expectativa de vida ao longo dos anos
iraq = dataset[dataset['country'] == 'Iraq']
iraq = iraq.sort_values(by='year')
iraq[['year', 'life_expectancy']]
plt.figure(figsize=(10, 5))
plt.xticks(np.arange(2000, 2016, 1.0))
plt.title('Expectativa de vida no Iraque ao longo dos anos')
plt.plot(iraq['year'], iraq['life_expectancy'], color = 'green')
plt.xlabel('Ano')
plt.ylabel('Expectativa de vida em anos')

Vemos a partir do gráfico que de 2000 a 2007 houve uma queda na expectativa de vida, mas não muito alta (aproximadamente 1 ano). Com a guerra chegando ao fim, a expectativa de vida voltou a subir, indicando que provavelmente a queda anterior foi devido à guerra.

 A seguir analisaremos as métricas relacionadas à mortalidade nesse período.

In [ ]:
#mortalidade infantil ao longo dos anos
plt.figure(figsize=(10, 5))
plt.xticks(np.arange(2000, 2016, 1.0))
plt.title('Mortalidade infantil no Iraque ao longo dos anos')
plt.plot(iraq['year'], iraq['under_five_deaths'], color = 'green')
plt.xlabel('Ano')
plt.ylabel('Mortes de crianças menores de 5 anos a cada 1000 habitantes')


É observável que não houve um aumento de mortes infantis no período observado, muito provavelmente por não se tratar de uma guerra civil, como foi o caso da Síria.

In [ ]:
#mortalidade adulta ao longo dos anos
plt.figure(figsize=(10, 5))
plt.xticks(np.arange(2000, 2016, 1.0))
plt.title('Mortalidade adulta no Iraque ao longo dos anos')
plt.plot(iraq['year'], iraq['adult_mortality'], color = 'green')
plt.xlabel('Ano')
plt.ylabel('Mortes de adultos a cada 1000 habitantes')


14 de Agosto de 2007 ocorreu o ataque mais mortífero desde o início da guerra. Mais de 500 civis foram mortos numa série de ataques bombistas suicidas coordenados no norte do Iraque em Qahtaniya. Mais de 100 casas e lojas foram destruídas nas explosões.

O formato desse gráfico, juntamente com o texto acima, nos faz deduzir que 2007 foi o ano mais intenso da guerra.
A diferença, no entanto, foi de aproximadamente 30 pessoas (a cada 1000), o que pode parecer não tão grande, mas devemos levar em conta que o número mínimo que temos de mortes já é bem alto. 
Além disso, é possível observar que, a partir de 2003, a inclinação da reta aumentou, alinhando com o início da guerra.

### COMPARANDO
Analiseremos como a guerra afetou diferentes aspectos nos dois países e as diferenças entre eles.

ESCOLARIDADE

In [ ]:
# grafico de schooling ao longo dos anos para a síria e o iraque
plt.figure(figsize=(10, 5))
plt.xticks(np.arange(2000, 2016, 1.0))
plt.title('Escolaridade na Síria e no Iraque ao longo dos anos')
plt.plot(syria['year'], syria['schooling'])
plt.plot(iraq['year'], iraq['schooling'], color = 'green')
plt.legend(['Síria', 'Iraque'])
plt.xlabel('Ano')
plt.ylabel('Média de anos passados estudando')

Comparando a taxa de escolaridade entre os dois países, vemos que a Síria, até 2010, tinha uma taxa de crescimento maior que a do Iraque, mas a partir de 2010, a taxa de crescimento da Síria caiu drasticamente, enquanto a do Iraque continuou a crescer. Isso pode ser explicado pelo fato de que a guerra civil na Síria começou em 2011, enquanto a Guerra do Iraque terminou em 2011. Além disso, a guerra civil na Síria pode ter afetado a educação no país, enquanto a Guerra do Iraque não afetou a educação no Iraque, muito provavelmente por não se tratar de uma guerra civil.

ECONOMIA

In [ ]:
#grafico com GPD per capita ao longo dos anos iraque e siria e media  mundial  
plt.figure(figsize=(10, 5))
plt.xticks(np.arange(2000, 2016, 1.0))
plt.title('GDP per capita Síria e no Iraque ao longo dos anos')
plt.plot(syria['year'], syria['gdp_per_capita'])
plt.plot(iraq['year'], iraq['gdp_per_capita'], color = 'green')
plt.legend(['Síria', 'Iraque'])
plt.xlabel('Ano')
plt.ylabel('GDP per capita em dólares')


IRAQUE: É possível observar uma enorme queda no PIB per capita em 2003, ano da invasão estadunidense que gerou o início da guerra, de mais de 1000 dólares. Em 2004 o PIB já havia se recuperado consideravelmente, mas só voltou ao patamar de 2002 em 2008. A partir de 2011, ano em que a Guerra do Iraque terminou, o PIB per capita começou a subir mais rapidamente. Por fim é possível observar que a diferença de 2000 à 2011 (ano de início e fim da guerra), não parece ser muito expressiva, mas representa a falta de crescimento na economia por 11 anos.

SÍRIA: É possível observar uma queda no PIB per capita em 2011, ano de início da guerra. A partir de 2013, houve uma estagnação, alinhada com o ano em que a mortalidade infantil e adulta começaram a cair. Isso pode indicar que de 2011 a 2013 a guerra estava em seu auge, e a partir de 2013 os impactos da guerra começaram a diminuir. Diferentemente do Iraque, não houve um crecimento no PIB per capita após a queda, indicando que a guerra afetou mais intensamente a economia da Síria.

Analisando numericamente o PIB do Iraque, complementando a análise acima.

In [ ]:
# diferenca de GDP per capita do iraque de 2002 e 2003
iraq[iraq['year'] == 2003]['gdp_per_capita'].values[0] - iraq[iraq['year'] == 2002]['gdp_per_capita'].values[0]
print('Diferença de GDP per capita do Iraque de 2002 a 2003: ', iraq[iraq['year'] == 2003]['gdp_per_capita'].values[0] - iraq[iraq['year'] == 2002]['gdp_per_capita'].values[0])


# diferenca de GDP per capita do iraque de 2003 e 2004
iraq[iraq['year'] == 2004]['gdp_per_capita'].values[0] - iraq[iraq['year'] == 2003]['gdp_per_capita'].values[0]
print('Diferença de GDP per capita do Iraque de 2003 a 2004: ', iraq[iraq['year'] == 2004]['gdp_per_capita'].values[0] - iraq[iraq['year'] == 2003]['gdp_per_capita'].values[0])

# diferenca de GDP per capita do iraque de 2002 e 2004
iraq[iraq['year'] == 2004]['gdp_per_capita'].values[0] - iraq[iraq['year'] == 2002]['gdp_per_capita'].values[0]
print('Diferença de GDP per capita do Iraque de 2002 a 2004: ', iraq[iraq['year'] == 2004]['gdp_per_capita'].values[0] - iraq[iraq['year'] == 2002]['gdp_per_capita'].values[0])

#diferenca de GDP per capita do iraque de 2002 e 2008
iraq[iraq['year'] == 2008]['gdp_per_capita'].values[0] - iraq[iraq['year'] == 2002]['gdp_per_capita'].values[0]
print('Diferença de GDP per capita do Iraque de 2002 a 2008: ', iraq[iraq['year'] == 2008]['gdp_per_capita'].values[0] - iraq[iraq['year'] == 2002]['gdp_per_capita'].values[0])

# diferenca de GDP per capita do iraque de 2000 e 2011
iraq[iraq['year'] == 2011]['gdp_per_capita'].values[0] - iraq[iraq['year'] == 2000]['gdp_per_capita'].values[0]
print('Diferença de GDP per capita do Iraque de 2000 a 2011: ', iraq[iraq['year'] == 2011]['gdp_per_capita'].values[0] - iraq[iraq['year'] == 2000]['gdp_per_capita'].values[0])


Até então, com os dados observados, tudo parece indicar que a guerra da Síria foi mais devastadora que a do Iraque, em vários aspectos. 

POPULAÇÃO

In [ ]:
#grafico de populacao ao longo dos anos para a síria e o iraque
plt.figure(figsize=(10, 5))
plt.xticks(np.arange(2000, 2016, 1.0))
plt.title('População Síria e no Iraque ao longo dos anos')
plt.plot(syria['year'], syria['population_mln'])
plt.plot(iraq['year'], iraq['population_mln'], color = 'green')
plt.legend(['Síria', 'Iraque'])
plt.xlabel('Ano')
plt.ylabel('População em milhões')

Mais uma vez, a Síria parece ter sido mais afetada pela guerra que o Iraque, com a população diminuindo a partir de 2010, se alinhando com o início da guerra civil. No Iraque, a população continuou a crescer, mesmo durante a guerra. A inclinação da reta do Iraque, a partir de 2010, se tornou mais acentuada, indicando que a população cresceu mais rápido a partir de 2010. Um dos fatores que pode ter contribuído para isso é o fato de que a guerra acabou em 2011. Agora compararemos esse crescimento com a média mundial.

In [ ]:
media_populacao = dataset.groupby('year')['population_mln'].mean()

plt.figure(figsize=(10, 5))
plt.xticks(np.arange(2000, 2016, 1.0))
plt.title('População Síria e no Iraque ao longo dos anos, comparado com a média mundial')
plt.plot(syria['year'], syria['population_mln'])
plt.plot(iraq['year'], iraq['population_mln'], color = 'green')
plt.plot(media_populacao, color = 'red')
plt.legend(['Síria', 'Iraque', 'Média mundial'])
plt.xlabel('Ano')
plt.ylabel('População em milhões')

A partir desse gráfico, é possível concluir que a população do Iraque teve um crescimento bem expressivo desde 2010, comparando com a média mundial.

Após todas as análises, podemos concluir as guerras afetaram os dois países de maneiras diferentes, que o Iraque se recompos bem após a guerra, e que, comparado com a Síria, a guerra afetou menos o país, mas ainda assim houveram repercussões, principalmente no que tange a economia. 

Um importante fator que não pode ser deixado de lado é o fato de que, durante uma guerra, a coleta de dados é muito mais difícil, e muitas vezes não é possível coletar dados de forma precisa, o que pode ter afetado os resultados obtidos. Então os resultados correspodem ao que foi possível coletar, mas não necessariamente refletem a realidade.


### ÁFRICA

In [ ]:

# grafico com porcentagem de diferenca de populacao entre 2000 e 2015 por regiao

plt.xticks(np.arange(2000, 2016, 1.0))
plt.title('Porcentagem de diferença de população entre 2000 e 2015 por região')
region_unique = dataset['region'].unique()
for region in region_unique:
    region_pop = dataset[dataset['region'] == region]
    region_pop = region_pop.groupby('year')['population_mln'].sum()
    region_pop = (region_pop[2015] - region_pop[2000]) / region_pop[2000] * 100
    plt.bar(region, region_pop)
plt.legend(region_unique)
plt.xlabel('Região')
plt.ylabel('Porcentagem de aumento de população')

A partir do gráfico acima, observamos que entre todas as regiões, a África possui o maior aumento populacional quando comparadas as populações de 2000 e 2015. A seguir vamos investigar possíveis causas para esse aumento tão expressivo.

#### HIV

O HIV/AIDS é um importante problema de saúde pública e causa milhões de mortes em muitas partes da África. A África subsaariana sozinha respondeu por cerca de 69% de todas as pessoas que viviam com o HIV e 70% de todas as mortes causadas pela doença em 2011.

In [ ]:
#hiv africa ao longo dos anos
africa = dataset[dataset['region'] == 'Africa']
africa = africa.groupby('year')['incidents_hiv'].mean()
plt.figure(figsize=(10, 5))
plt.xticks(np.arange(2000, 2016, 1.0))
plt.title('Incidentes de HIV na África ao longo dos anos')
plt.plot(africa)
plt.xlabel('Ano')
plt.ylabel('Incidentes de HIV (a cada 1000 pessoas)')

In [ ]:
#diferença entre casos de HIV 2000 e 2015
africa = dataset[dataset['region'] == 'Africa']
africa = africa.groupby('year')['incidents_hiv'].mean()
#casos de hiv 2000 e 2015
print('Incidentes de HIV na Africa em 2000: ', africa[2000])
print('Incidentes de HIV na Africa em 2015: ', africa[2015])

print('Diferença entre incidentes de HIV 2000 e 2015: ', africa[2000] - africa[2015])


In [ ]:
# hiv por regiao ao longo dos anos
plt.figure(figsize=(10, 8))
plt.xticks(np.arange(2000, 2016, 1.0))
plt.title('Incidentes de HIV por região ao longo dos anos')

region_unique = dataset['region'].unique()
for region in region_unique:
    region_hiv = dataset[dataset['region'] == region]
    region_hiv = region_hiv.groupby('year')['incidents_hiv'].mean()
    plt.plot(region_hiv)

plt.legend(region_unique, loc='upper right')
plt.xlabel('Ano')
plt.ylabel('Incidentes de HIV (a cada 1000 pessoas)')


A partir das análises acima, varias conclusões podem ser tomadas
- A África é a região com maior proporção de pessoas infectadas pelo HIV, com uma diferença de mais de 3 pessoas a cada 1000 em relação à segunda região com maior proporção de infectados em 2000.
- A diferença de infectados entre 2000 e 2015 na África é bem significativa, com uma queda de mais de 2 pessoas a cada 1000.
- Em 2015, a África ainda é a região com maior proporção de infectados, mas a diferença entre ela e a segunda região com maior número de infectados é bem menor que em 2000

Tudo isso indica que a diminuição de incidência HIV foi bem expressiva, e pode ter contribuído para o aumento populacional na região.

#### DOENÇAS

In [ ]:
#taxa de cobertura de vacinação em bebes de 1 ano de doenças na africa ao longo dos anos
africa = dataset[dataset['region'] == 'Africa']
measles = africa.groupby('year')['measles'].mean()
polio = africa.groupby('year')['polio'].mean()
diphtheria = africa.groupby('year')['diphtheria'].mean()
hepatite_b = africa.groupby('year')['hepatitis_b'].mean()
plt.figure(figsize=(10, 5))
plt.xticks(np.arange(2000, 2016, 1.0))
plt.title('Taxa de vacinação de doenças na África ao longo dos anos')
plt.plot(measles)
plt.plot(polio)
plt.plot(diphtheria)
plt.plot(hepatite_b)
plt.legend(['Sarampo', 'Polio', 'Difteria', 'Hepatite B'])
plt.xlabel('Ano')
plt.ylabel('Taxa de cobertura de vacinação em bebês de 1 ano')

A taxa de vacinação de crianças é um importante indicador de saúde pública. É visível que a vacinação contra Polio, Difteria e Hepatite B aumentou consideralvemente na região, o que pode ter contribuído para a diminuição da mortalidade infantil, e consequentemente, para o aumento populacional.
Já a taxa da Sarampo não teve uma variação tão grande, e é expressivamente menor que as outras, o que pode indicar que a doença não é tão prevalente na região, ou que a vacinação não é tão disseminada.

Durante os anos de 2003 e 2004 ocorreu uma pequena queda de crescimento da taxa de imuninazação de Pólio, que pode ser explicada pois, na região norte, de maioria muçulmana, a pressão dos círculos salafistas interrompeu as campanhas de vacinação contra a poliomelite. Um boato afirmava que eram uma ferramenta de um grande complô internacional para esterilizar os muçulmanos. 

Agora, comparando com as taxas do resto do mundo

In [ ]:

resto_do_mundo = dataset[dataset['region'] != 'Africa']
measles = resto_do_mundo.groupby('year')['measles'].mean()
polio = resto_do_mundo.groupby('year')['polio'].mean()
diphtheria = resto_do_mundo.groupby('year')['diphtheria'].mean()
hepatite_b = resto_do_mundo.groupby('year')['hepatitis_b'].mean()
africa = dataset[dataset['region'] == 'Africa']
africa_measles = africa.groupby('year')['measles'].mean()
africa_polio = africa.groupby('year')['polio'].mean()
africa_diphtheria = africa.groupby('year')['diphtheria'].mean()
africa_hepatite_b = africa.groupby('year')['hepatitis_b'].mean()
plt.figure(figsize=(10, 5))
plt.xticks(np.arange(2000, 2016, 1.0))
plt.title('Taxa de vacinação de doenças no mundo e na África ao longo dos anos')
plt.plot(measles, color = 'purple')
plt.plot(polio, color = 'orange')
plt.plot(diphtheria, color = 'green')
plt.plot(hepatite_b, color = 'red')
plt.plot(africa_measles, linestyle='dashed', color = 'purple')
plt.plot(africa_polio, linestyle='dashed', color = 'orange')
plt.plot(africa_diphtheria, linestyle='dashed', color = 'green')
plt.plot(africa_hepatite_b, linestyle='dashed', color = 'red')
plt.legend(['Sarampo', 'Polio', 'Difteria','Hepatite B','Sarampo África', 'Pólio África', 'Difteria África', 'Hepatite B África'])
plt.xlabel('Ano')
plt.ylabel('Taxa de cobertura de vacinação em bebês de 1 ano')

- É possivel observar que a taxa de vacinação na África, mesmo apresentando um grande crescimento, ainda é bem menor que a média mundial, o que pode ser explicado pelo fato de que a África é um continente mais pobre, que a vacinação é um processo caro e que o acesso a muitas regiões é dificultado por guerras, conflitos e outros fatores.

- Também vemos que a imunização contra sarampo, em todo o mundo, incluindo a África, é bem menor que a imunização contra polio e difteria e hepatite B.

- Por fim, vemos que, na Africa, até 2007, a vacinação contra Hepatite B era maior que a vacinação contra as outras doenças, e que depois disso as taxas de vacinação contra polio e difteria tenderam a se igualar com a de hepatite B. No entanto, no mundo, de 2000 a 2015, a vacinação contra polio e difteria sempre foi maior que a vacinação contra hepatite B.

#### MORTALIDADE 
##### BEBÊS E CRIANÇAS

In [ ]:
# mortalidade de bebês, crianças e adultos na africa e no mundo ao longo dos anos
resto_do_mundo = dataset[dataset['region'] != 'Africa']
resto_do_mundo_bebes = resto_do_mundo.groupby('year')['infant_deaths'].mean()
resto_do_mundo_criancas = resto_do_mundo.groupby('year')['under_five_deaths'].mean()
africa = dataset[dataset['region'] == 'Africa']
africa_bebes = africa.groupby('year')['infant_deaths'].mean()
africa_criancas = africa.groupby('year')['under_five_deaths'].mean()
plt.figure(figsize=(10, 5))
plt.xticks(np.arange(2000, 2016, 1.0))
plt.title('Mortalidade de bebês e crianças no mundo e na África ao longo dos anos')
plt.plot(resto_do_mundo_bebes, color = 'purple')
plt.plot(resto_do_mundo_criancas, color = 'orange')
plt.plot(africa_bebes, linestyle='dashed', color = 'purple')
plt.plot(africa_criancas, linestyle='dashed', color = 'orange')
plt.legend(['Bebês', 'Crianças', 'Bebês Africa', 'Crianças Africa'])
plt.xlabel('Ano')
plt.ylabel('Mortes (a cada 1000 pessoas)')

A partir do gráfico acima, é possível observar que a mortalidade infantil na África é extremamente maior que a média mundial, mas também que ela vem diminuindo ao longo dos anos, e um dos fatores que pode ter contribuído para isso é a vacinação, como visto anteriormente.


##### ADULTOS

In [ ]:
#mortalidade adulta africa e resto do mundo
resto_do_mundo = dataset[dataset['region'] != 'Africa']
resto_do_mundo_adultos = resto_do_mundo.groupby('year')['adult_mortality'].mean()
africa = dataset[dataset['region'] == 'Africa']
africa_adultos = africa.groupby('year')['adult_mortality'].mean()
plt.figure(figsize=(10, 5))
plt.xticks(np.arange(2000, 2016, 1.0))
plt.title('Mortalidade de adultos no mundo e na África ao longo dos anos')
plt.plot(resto_do_mundo_adultos, color = 'purple')
plt.plot(africa_adultos, linestyle='dashed', color = 'purple')
plt.legend(['Adultos', 'Adultos Africa'])
plt.xlabel('Ano')
plt.ylabel('Mortes (a cada 1000 pessoas)')

Assim como a mortalidade infantil, a mortalidade de adultos na África é bem maior que a média mundial, com uma diferença de aproximadamente 200 adultos durante todos os anos, mas também vem diminuindo ao longo dos anos.


In [ ]:
#maior mortalidade de adultos na africa 
print('Maior mortalidade de adultos na África: ', africa_adultos.max())
#media de mortalidade de adultos na africa
print('Média de mortalidade de adultos na África: ', africa_adultos.mean())



Comparando com a mortalidade infantil durante guerras


In [ ]:
#africa e siria de mortes infantis por ano
africa = dataset[dataset['region'] == 'Africa']
africa = africa.groupby('year')['under_five_deaths'].mean()
syria = dataset[dataset['country'] == 'Syrian Arab Republic']
syria = syria.groupby('year')['under_five_deaths'].mean()
plt.figure(figsize=(10, 5))
plt.xticks(np.arange(2000, 2016, 1.0))
plt.title('Mortalidade infantil na África e Síria ao longo dos anos')
plt.plot(africa)
plt.plot(syria)
plt.legend(['África', 'Síria'])
plt.xlabel('Ano')
plt.ylabel('Mortes (a cada 1000 pessoas)')


É bem interessante por em persepectiva com a mortalidade infantil da síria, que durante a guerra civil aumentou exorbitantemente, pois, mesmo com esse aumento, e com a mortalidade infantil na África tendo diminuido muito, a mortalidade infantil na África ainda é maior que a da Síria, o que mostra que a situação no continente é bem precária e que ainda há muito a ser feito, já que um país em guerra, com bombardeios, destruição e falta de recursos, ainda tem taxas menos piores que a África.

# Regressão Linear Simples

Nosso objetivo é prever a média da expectativa de vida ao redor do mundo em 2015, com base nos dados de 2000 a 2014. Para isso, vamos utilizar um modelo de regressão linear considerando expectativa de vida por ano.

O primeiro plot nos mostra a expectativa de vida ao longo dos anos por região, e as retas de regressão linear para cada região, que parecem se adaptar bem aos dados.

In [ ]:
regions = dataset['region'].unique()

for region in regions:
    region_data = dataset[dataset['region'] == region]
    for year in range(2000, 2016):
        region_data.loc[region_data['year'] == year, 'life_expectancy'] = region_data[region_data['year'] == year]['life_expectancy'].mean()
    # um grafico do lado do
    sns.regplot(x='year', y='life_expectancy', data=region_data, n_boot=10000,
                line_kws={'color':'magenta', 'lw':2},
                scatter_kws={'edgecolor':'k', 's':80, 'alpha':0.8})
    # colocar regiao como legenda
    plt.title('Expectativa de vida ao longo dos anos por região')
    plt.xlabel('Ano')
    plt.ylabel('Expectativa de vida')

    

    
plt.show()


Calcularemos agora os parâmetros da regressão linear:

In [ ]:
# copia o dataset
df = dataset.copy()

for year in range(2000, 2016):
    df.loc[df['year'] == year, 'life_expectancy'] = df[df['year'] == year]['life_expectancy'].mean() # media da expectativa de vida global por ano

# calcula a regressao linear simples
slope, intercept, r_value, p_value, std_err = stats.linregress(df['year'], df['life_expectancy'])

# printa os resultados e plota o grafico
print('R²: ', r_value**2)
print('Beta: ', slope)
print('Intercepto: ', intercept)
#print('P-value: ', p_value) p_value do que??
#print('Std error: ', std_err)
print('')

# comparar valor previsto da expectativa de vida em 2015 com o valor real
print('Valor previsto da expectativa de vida em 2015: ', intercept + slope*2015)
print('Valor real da expectativa de vida em 2015: ', df[df['year'] == 2015]['life_expectancy'].mean())

# diferenca entre o valor previsto e o valor real
print('Diferença entre o valor previsto e o valor real: ', intercept + slope*2015 - df[df['year'] == 2015]['life_expectancy'].mean())



O valor previsto se aproxima muito da expectativa de vida real, e, considerando um R² de 0.99, podemos concluir que o modelo se ajustou muito bem aos dados, como pode ser visto pelo plot abaixo:

In [ ]:
plt.figure(figsize=(10, 5))
plt.xticks(np.arange(2000, 2016, 1.0))
plt.title('Expectativa de vida ao longo dos anos')
plt.xlabel('Ano')
plt.ylabel('Expectativa de vida')
plt.plot(df['year'], df['life_expectancy'], 'o', label='Dados originais')
plt.plot(df['year'], intercept + slope*df['year'], 'r', label='Linha de regressão')
plt.legend()
plt.show()

# Regressão Linear Múltipla

Nosso objetivo é prever a expectativa de vida do Brasil entre 2000 e 2015 com base em um modelo que leva em consideração todos os países do dataset (menos o Brasil). Para isso, utilizaremos a regressão linear múltipla. 

Antes de calcular a regressão múltipla, calcularemos a correlação entre cada atributo e a expectativa de vida. Assim, poderemos selecionar os atributos com maior correlação para fazer a regressão linear múltipla.

In [ ]:
atributos = ['life_expectancy', 'infant_deaths', 'under_five_deaths', 'adult_mortality', 'alcohol_consumption', 'hepatitis_b', 'measles', 'bmi', 'polio', 'diphtheria', 'incidents_hiv', 'gdp_per_capita', 'population_mln', 'schooling']

# tirar media de cada atributo por ano e colocar em um novo dataset
df_reg = dataset.copy()

# calcula a matriz de correlacao
corr = df_reg[atributos].corr()

# plota a matriz de correlacao da expectativa de vida
plt.figure(figsize=(10, 5))
plt.title('Matriz de correlação da expectativa de vida')
sns.heatmap(corr[['life_expectancy']].sort_values(by='life_expectancy', ascending=False), vmin=-1, vmax=1, annot=True, cmap='BrBG')
plt.show()



Analisando as correlações, concluímos que o consumo de álcool e a população não estão correlaciondos com a expectativa de vida, então podemos retirar essas variáveis da análise.

Porém, antes de eliminar as variáveis e z-normalizar os dados, iremos separar os dados relativos ao Brasil, para futura comparação.

In [ ]:
# retira os valores do brasil do dataset
df_brasil = df_reg[df_reg['country'] == 'Brazil']
df_reg = df_reg[df_reg['country'] != 'Brazil']

In [ ]:
# drop de atributos pouco correlacionados ou que não fazem sentido para a regressão linear múltipla
df_reg = df_reg.drop(columns=['alcohol_consumption', 'country', 'region', 'year', 'developed', 'population_mln'])

In [ ]:
# z-normalizar os dados valorados
df_reg = (df_reg - df_reg.mean()) / df_reg.std()
df_reg.head()

Definimos aqui as funções que serão utilizadas para calcular a regressão linear múltipla, como a função que retorna a jacobiana e a função que calcula o gradiente.

In [ ]:
# define a função que calcula a jacobiana
def jacobiana(theta, X, y):
    return -2 * ((y - X @ theta) * X.T).mean(axis=1)

# define a funcao de gradiente descendente
def gd(theta, X, y, lambda_=0.01, tol=0.00001, max_iter=10000):
    theta = theta.copy()
    old_err_sq = np.inf
    i = 0
    while True:
        # Computar as derivadas
        grad = jacobiana(theta, X, y)
        # Atualizar
        theta_novo = theta - lambda_ * grad
        
        # Parar quando o erro convergir
        err_sq = ((X.dot(theta) - y) ** 2).mean()
        if np.abs(old_err_sq - err_sq) <= tol:
            break
        theta = theta_novo
        old_err_sq = err_sq
        i += 1
        if i == max_iter:
            break
    return theta


# define a funcao de regressao linear multipla
def regressao_linear_multipla(X, y):
    theta = np.ones(X.shape[1])

    return gd(theta, X, y)


# printa os resultados da regressao linear multipla
y = df_reg['life_expectancy'].values
X = df_reg.drop(columns=['life_expectancy']).values
 
# adiciona o intercepto 1 ao X
X = np.c_[np.ones(X.shape[0]), X]

theta = regressao_linear_multipla(X, y)
print('Theta calculado pela nossa regressão linear múltipla: ', theta)

# analisar se os resultados batem com o scikit learn
from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=True)
model.fit(X, y)

LinearRegression()
model.coef_

O Gradiente Descendente aplicado com o método _gd_ que aprendemos em sala se aproxima do resultado obtido com o scikit learn.

Vamos calcular o R² para avaliar o modelo.

In [ ]:
# calcula o R² a partir do y real e do y previsto
y_pred = X @ theta
r2 = 1 - ((y - y_pred)**2).sum() / ((y - y.mean())**2).sum()
print('R²: ', r2)

# R² do scikit learn
print('R² do scikit learn: ', model.score(X, y))

O R² de 0.97 indica que o modelo se ajustou bem aos dados.

## Prever a expectativa de vida do Brasil com o modelo

Utilizaremos o modelo para prever a expectativa de vida do Brasil, entre 2000 e 2015, de acordo com os atributos considerados anteriormente.

In [ ]:
# a partir dos dados do brasil, prever a expectativa de vida com a regressao linear multipla calculada
df_brasil = df_brasil.drop(columns=['alcohol_consumption', 'country', 'region', 'year', 'developed', 'population_mln'])
dataset2 = dataset.drop(columns=['alcohol_consumption', 'country', 'region', 'year', 'developed', 'population_mln'])

exp_vida_brasil_real = df_brasil['life_expectancy'].values

# z-normalizar o df_brasil com base no dataset2
df_brasil = (df_brasil - dataset2.mean()) / dataset2.std(numeric_only=True)
y_brasil = df_brasil['life_expectancy'].values
X_brasil = df_brasil.drop(columns=['life_expectancy']).values

In [ ]:
# adiciona o intercepto 1 ao X
X_brasil = np.c_[np.ones(X_brasil.shape[0]), X_brasil]

# calcula a expectativa de vida prevista para o Brasil com os pesos calculados pela regressao linear multipla
expectativa_vida_prevista = X_brasil @ theta

# voltando a expectativa de vida prevista para a escala original
expectativa_vida_prevista = expectativa_vida_prevista * dataset['life_expectancy'].std() + dataset['life_expectancy'].mean()
y_brasil = y_brasil * dataset['life_expectancy'].std() + dataset['life_expectancy'].mean()

print('Expectativa de vida prevista para o Brasil: ', expectativa_vida_prevista)
print('\nExpectativa de vida real para o Brasil: ', exp_vida_brasil_real)

# calcula a diferenca entre a expectativa de vida prevista e a real
diferenca = expectativa_vida_prevista - y_brasil
print('\nDiferenca entre a expectativa de vida prevista e a real: ', diferenca)

print('\nMédia da diferença: ', diferenca.mean())

A diferença entre a expectativa de vida prevista e a real é de aproximadamente -1 ano e meio, o que é um erro considerável, mas não tão grande, considerando que estamos prevendo a expectativa de vida de um país inteiro, com base em dados de outros países, sendo que muitos desses têm expecativas de vida menores que a do Brasil.

In [ ]:
#plota a expectativa de vida real do brasil do dataset original
plt.figure(figsize=(10, 5))
#pontos reais
plt.scatter(dataset[dataset['country'] == 'Brazil']['year'], dataset[dataset['country'] == 'Brazil']['life_expectancy'], color='r', label='Expectativa de vida real')

#pontos previstos
plt.scatter(dataset[dataset['country'] == 'Brazil']['year'], expectativa_vida_prevista, color='b', label='Expectativa de vida prevista')
plt.title('Expectativa de vida real do Brasil')
plt.xlabel('Ano')
plt.ylabel('Expectativa de vida')
plt.legend()
plt.show()

## Avaliando os atributos do modelo

Agora avaliaremos a importância de cada atributo para o nosso modelo. Isso será feito por meio do método de _bootstrap_.

In [ ]:
def bootstrap_regression(X, y, n=10000, size=None):
    if size is None:
        size = len(df)
        
    T = np.zeros(shape=(n, X.shape[1]))
    for i in range(n):
        # Gera amostras com reposição
        idx = np.random.choice(len(y), len(y))
        Xb = X[idx]
        yb = y[idx]
        
        # Fit usando sklearn
        model = LinearRegression(fit_intercept=True)
        model.fit(Xb, yb)
        
        T[i] = model.coef_
    return T


y = dataset['life_expectancy'].values
attributes = ['infant_deaths', 'under_five_deaths', 'adult_mortality', 'hepatitis_b', 'measles', 'bmi', 'polio', 'diphtheria', 'incidents_hiv', 'gdp_per_capita', 'schooling']
X = dataset[attributes].values
T = bootstrap_regression(X, y)

for col in range(X.shape[1]):
    plt.hist(T[:, col], edgecolor='k')
    plt.title('{} - 95% CI Bootstrap({:.2f}, {:.2f})'.format(attributes[col],
                                                             np.percentile(T[:, col], 2.5),
                                                             np.percentile(T[:, col], 97.5)))
    plt.xlabel(r'$\theta_i$')
    plt.ylabel('# Boot Amostras')
    plt.show()

Concluímos que alguns atributos apresentam o valor zero no Intervalo de Confiança, logo, não são relevantes para o modelo. São eles: measles, polio, diphteria e incidents_hiv.